# 03 — Downscaling & Evaluation

Fit EQM on training period, apply to test or target period, evaluate skill, and plot.

In [ ]:

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from src.downscaling import empirical_quantile_mapping
from src.utils import mae, bias, nse, monthly_climatology

# Load matched datasets
pr_cmip = xr.open_dataarray('data/processed/cmip6_ea_matched.nc')
obs = xr.open_dataset('data/processed/obs_ea_matched.nc')
obs_pr = obs['pr'] if 'pr' in obs else list(obs.data_vars.values())[0]


In [ ]:

# Choose train/test split
train_slice = slice('1981-01-01', '2005-12-31')
test_slice  = slice('2006-01-01', '2014-12-31')

sim_train = pr_cmip.sel(time=train_slice)
obs_train = obs_pr.sel(time=train_slice)
sim_test  = pr_cmip.sel(time=test_slice)
obs_test  = obs_pr.sel(time=test_slice)
sim_train, obs_train, sim_test, obs_test


In [ ]:

# Fit & apply EQM (additive by default; for precipitation you can try kind='MUL')
pr_down = empirical_quantile_mapping(sim_train, obs_train, sim_test, kind='MUL', nquantiles=80, group='time.month')
pr_down.name = 'pr_downscaled'
pr_down.to_netcdf('data/processed/pr_downscaled_test.nc')
pr_down


In [ ]:

# Evaluate metrics over space/time
metrics = {
    'MAE': mae(pr_down, obs_test, dim='time').mean(['lat','lon']).item(),
    'Bias': bias(pr_down, obs_test, dim='time').mean(['lat','lon']).item(),
    'NSE': nse(pr_down, obs_test, dim='time').mean(['lat','lon']).item(),
}
metrics


In [ ]:

# Timeseries: EA spatial mean
ts_obs = obs_test.mean(['lat','lon'])
ts_raw = sim_test.mean(['lat','lon'])
ts_down = pr_down.mean(['lat','lon'])

plt.figure()
ts_obs.plot(label='Obs')
ts_raw.plot(label='Raw CMIP6')
ts_down.plot(label='Downscaled')
plt.legend(); plt.title('East Africa mean daily precipitation')
plt.savefig('figures/plots/ea_timeseries.png', dpi=150)


In [ ]:

# Climatology comparison
clim_obs = monthly_climatology(obs_test).mean(['lat','lon'])
clim_raw = monthly_climatology(sim_test).mean(['lat','lon'])
clim_down = monthly_climatology(pr_down).mean(['lat','lon'])

plt.figure()
clim_obs.plot(marker='o', label='Obs')
clim_raw.plot(marker='o', label='Raw CMIP6')
clim_down.plot(marker='o', label='Downscaled')
plt.legend(); plt.title('Monthly climatology (EA mean)')
plt.savefig('figures/plots/ea_climatology.png', dpi=150)
